# Bike Sharing Demand 02 - TensorFlow deeper

###   
### 목표 : 텐서플로를 활용하여 은닉층, 손실함수, 활성화 함수 등의 조건을 달리해 보아 예측 모델을 개발한다. 
###   
###   

## 1. 데이터 전처리 

In [1]:
import pandas as pd
import numpy as np

train = pd.read_csv("./bike/train.csv", parse_dates=['datetime'])
test = pd.read_csv("./bike/test.csv", parse_dates=['datetime'])

train['year'] = train['datetime'].dt.year  
train['month'] = train['datetime'].dt.month
train['day'] = train['datetime'].dt.day
train['hour'] = train['datetime'].dt.hour
train['minute'] = train['datetime'].dt.minute
train['second'] = train['datetime'].dt.second
train["dayofweek"] = train["datetime"].dt.dayofweek 

test['year'] = test['datetime'].dt.year  
test['month'] = test['datetime'].dt.month
test['day'] = test['datetime'].dt.day
test['hour'] = test['datetime'].dt.hour
test['minute'] = test['datetime'].dt.minute
test['second'] = test['datetime'].dt.second
test["dayofweek"] = train["datetime"].dt.dayofweek 

train["season_1"] = train["season"] == 1
train["season_2"] = train["season"] == 2
train["season_3"] = train["season"] == 3
train["season_4"] = train["season"] == 4

test["season_1"] = test["season"] == 1
test["season_2"] = test["season"] == 2
test["season_3"] = test["season"] == 3
test["season_4"] = test["season"] == 4

train["weather_1"] = train["weather"] == 1
train["weather_2"] = train["weather"] == 2
train["weather_3"] = train["weather"] == 3
train["weather_4"] = train["weather"] == 4

test["weather_1"] = test["weather"] == 1
test["weather_2"] = test["weather"] == 2
test["weather_3"] = test["weather"] == 3
test["weather_4"] = test["weather"] == 4

train["mon"] = train["dayofweek"] == 0
train["tue"] = train["dayofweek"] == 1
train["wed"] = train["dayofweek"] == 2
train["thu"] = train["dayofweek"] == 3
train["fri"] = train["dayofweek"] == 4
train["sat"] = train["dayofweek"] == 5
train["sun"] = train["dayofweek"] == 6

test["mon"] = test["dayofweek"] == 0
test["tue"] = test["dayofweek"] == 1
test["wed"] = test["dayofweek"] == 2
test["thu"] = test["dayofweek"] == 3
test["fri"] = test["dayofweek"] == 4
test["sat"] = test["dayofweek"] == 5
test["sun"] = test["dayofweek"] == 6

train["12 am"] = train["hour"] == 0 
test["12 am"] = test["hour"] == 0 

train["1 am"] = train["hour"] == 1
test["1 am"] = test["hour"] == 1 

train["2 am"] = train["hour"] == 2 
test["2 am"] = test["hour"] == 2 

train["3 am"] = train["hour"] == 3 
test["3 am"] = test["hour"] == 3 

train["4 am"] = train["hour"] == 4 
test["4 am"] = test["hour"] == 4 

train["5 am"] = train["hour"] == 5 
test["5 am"] = test["hour"] == 5 

train["6 am"] = train["hour"] == 6 
test["6 am"] = test["hour"] == 6 

train["7 am"] = train["hour"] == 7 
test["7 am"] = test["hour"] == 7 

train["8 am"] = train["hour"] == 8 
test["8 am"] = test["hour"] == 8

train["9 am"] = train["hour"] == 9 
test["9 am"] = test["hour"] == 9 

train["10 am"] = train["hour"] == 10 
test["10 am"] = test["hour"] == 10 

train["11 am"] = train["hour"] == 11 
test["11 am"] = test["hour"] == 11 

train["12 pm"] = train["hour"] == 12 
test["12 pm"] = test["hour"] == 12

train["1 pm"] = train["hour"] == 13 
test["1 pm"] = test["hour"] == 13 

train["2 pm"] = train["hour"] == 14 
test["2 pm"] = test["hour"] == 14 

train["3 pm"] = train["hour"] == 15 
test["3 pm"] = test["hour"] == 15

train["4 pm"] = train["hour"] == 16 
test["4 pm"] = test["hour"] == 16

train["5 pm"] = train["hour"] == 17 
test["5 pm"] = test["hour"] == 17

train["6 pm"] = train["hour"] == 18
test["6 pm"] = test["hour"] == 18

train["7 pm"] = train["hour"] == 19 
test["7 pm"] = test["hour"] == 19

train["8 pm"] = train["hour"] == 20
test["8 pm"] = test["hour"] == 20

train["9 pm"] = train["hour"] == 21
test["9 pm"] = test["hour"] == 21

train["10 pm"] = train["hour"] == 22
test["10 pm"] = test["hour"] == 22

train["11 pm"] = train["hour"] == 23
test["11 pm"] = test["hour"] == 23


feature_names = ['holiday', 'workingday', 'temp', 'atemp', 'humidity', 'windspeed',
                 'year', 'month', 'day', 'hour', 'minute', 'second',
                 'season_1', 'season_2', 'season_3', 'season_4',
                 'weather_1', 'weather_2', 'weather_3', 'weather_4',
                 'mon', 'tue', 'wed', 'thu', 'fri', 'sat', 'sun',
                 '12 am', '1 am', '2 am', '3 am', '4 am', '5 am', '6 am', '7 am', '8 am', '9 am', '10 am', '11 am', 
                 '12 pm', '1 pm', '2 pm', '3 pm', '4 pm', '5 pm', '6 pm', '7 pm', '8 pm', '9 pm', '10 pm', '11 pm',
                 ]

x_train = train[feature_names]
y_train = train['count']
x_test = test[feature_names]


y_train = np.log1p(y_train) # np.log(y_train + 1)

x_train_1 = np.array(x_train, dtype=np.float32)
y_train_1 = np.array(y_train, dtype=np.float32)
x_test_1 = np.array(x_test, dtype=np.float32)

x_train_1 = x_train_1.reshape(10886, 51)
y_train_1 = y_train_1.reshape(10886, 1)
x_test_1 = x_test_1.reshape(6493, 51)

#### 데이터를 텐서플로에서 사용할 수 있게 변환함.

####   
####   

## 2. TensorFlow 신경망 모델 구성

### 텐서플로우 실행

In [3]:
import tensorflow as tf

hello = tf.constant('hello, tensorflow!')
print(hello)

Tensor("Const_1:0", shape=(), dtype=string)


### 1) 입출력 정의

In [4]:
X = tf.placeholder(tf.float32, [None, 51]) 
Y = tf.placeholder(tf.float32, [None, 1])  
keep_prob = tf.placeholder(tf.float32)

### 2) 은닉층 구성

#### (1) 기존 은닉층

In [ ]:
W1 = tf.Variable(tf.random_normal([51 ,256], stddev = 0.01))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)


W2 = tf.Variable(tf.random_normal([256, 256], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([256]))
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([256, 1], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([1]))
model = tf.add(tf.matmul(L2, W3), b3)

#### 이 모델로 100만 번 학습시켰으나 성능이 더이상 향상되지 않음

#### (2) 은닉층 5개 구성

In [4]:
W1 = tf.Variable(tf.random_normal([51 ,256], stddev = 0.01))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)


W2 = tf.Variable(tf.random_normal([256, 512], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512, 512], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.add(tf.matmul(L2, W3), b3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([512, 256], stddev = 0.01))
b4 = tf.Variable(tf.random_normal([256]))
L4 = tf.add(tf.matmul(L3, W4), b4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.Variable(tf.random_normal([256, 1], stddev = 0.01))
b5 = tf.Variable(tf.random_normal([1]))
model = tf.add(tf.matmul(L4, W5), b5)

#### (3) 은닉층 7개 구성

In [5]:
W1 = tf.Variable(tf.random_normal([51 ,256], stddev = 0.01))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.relu(L1)
L1 = tf.nn.dropout(L1, keep_prob)


W2 = tf.Variable(tf.random_normal([256, 512], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.relu(L2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512, 512], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.add(tf.matmul(L2, W3), b3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([512, 512], stddev = 0.01))
b4 = tf.Variable(tf.random_normal([512]))
L4 = tf.add(tf.matmul(L3, W4), b4)
L4 = tf.nn.relu(L4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.Variable(tf.random_normal([512, 512], stddev = 0.01))
b5 = tf.Variable(tf.random_normal([512]))
L5 = tf.add(tf.matmul(L4, W5), b5)
L5 = tf.nn.relu(L5)
L5 = tf.nn.dropout(L5, keep_prob)

W6 = tf.Variable(tf.random_normal([512, 256], stddev = 0.01))
b6 = tf.Variable(tf.random_normal([256]))
L6 = tf.add(tf.matmul(L5, W6), b6)
L6 = tf.nn.relu(L6)
L6 = tf.nn.dropout(L6, keep_prob)

W7 = tf.Variable(tf.random_normal([256, 1], stddev = 0.01))
b7 = tf.Variable(tf.random_normal([1]))
model = tf.add(tf.matmul(L6, W7), b7)

#### (4) 은닉층 5개 구성 - 활성화 함수 쓰지 않기

In [5]:
W1 = tf.Variable(tf.random_normal([51 ,256], stddev = 0.01))
b1 = tf.Variable(tf.random_normal([256]))
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.dropout(L1, keep_prob)


W2 = tf.Variable(tf.random_normal([256, 512], stddev = 0.01))
b2 = tf.Variable(tf.random_normal([512]))
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_normal([512, 512], stddev = 0.01))
b3 = tf.Variable(tf.random_normal([512]))
L3 = tf.add(tf.matmul(L2, W3), b3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([512, 256], stddev = 0.01))
b4 = tf.Variable(tf.random_normal([256]))
L4 = tf.add(tf.matmul(L3, W4), b4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.Variable(tf.random_normal([256, 1], stddev = 0.01))
b5 = tf.Variable(tf.random_normal([1]))
model = tf.add(tf.matmul(L4, W5), b5)

#### (5) 은닉층 5개 구성 - 활성화 함수 쓰지 않기 - random_uniform

In [12]:
W1 = tf.Variable(tf.random_uniform([51 ,256]))
b1 = tf.Variable(tf.random_uniform([256]))
L1 = tf.add(tf.matmul(X, W1), b1)
L1 = tf.nn.dropout(L1, keep_prob)


W2 = tf.Variable(tf.random_uniform([256, 512]))
b2 = tf.Variable(tf.random_uniform([512]))
L2 = tf.add(tf.matmul(L1, W2), b2)
L2 = tf.nn.dropout(L2, keep_prob)

W3 = tf.Variable(tf.random_uniform([512, 512]))
b3 = tf.Variable(tf.random_uniform([512]))
L3 = tf.add(tf.matmul(L2, W3), b3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_uniform([512, 256]))
b4 = tf.Variable(tf.random_uniform([256]))
L4 = tf.add(tf.matmul(L3, W4), b4)
L4 = tf.nn.dropout(L4, keep_prob)

W5 = tf.Variable(tf.random_uniform([256, 1]))
b5 = tf.Variable(tf.random_uniform([1]))
model = tf.add(tf.matmul(L4, W5), b5)

### 3) 손실함수 및 최적화 

#### (1) 손실함수

#### cost = tf.reduce_mean(tf.square(model - Y))

In [6]:
cost = tf.reduce_mean(tf.square(model - Y))

#### 일단 가장 많이 쓰는 것 사용
#### 다른 손실함수도 찾아볼 것.

#### tf.sqrt(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))) 기존 작동되지 않았던 손실함수에 큰 실수 발견. sqrt는 루트지 제곱이 아님
#### 제곱인 square로 바꿔서 손실함수를 만든 후 모델을 돌려본다.

In [12]:
# cost = tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))))

In [14]:
# cost = tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))) 

In [5]:
# cost = tf.reduce_mean((tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))))

#### (2) 최적화 함수 

In [7]:
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

#### 다른 최적화 함수도 찾아서 써볼 것

### 4) 세션 열기

In [8]:
init = tf.global_variables_initializer()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

sess.run(init)

### 5) 학습 

In [9]:
import time
start = time.time()

for epoch in range(1000000):
    sess.run(optimizer, feed_dict = {X : x_train_1, Y: y_train_1, keep_prob : 0.7})
    
    if (epoch+1) % 100000 == 0:
        print((epoch + 1)/10000,"만 번 : ", sess.run(cost, feed_dict = {X : x_train_1, Y: y_train_1, keep_prob : 0.8}))
        imtime = time.time() - start
        m, s = divmod(imtime, 60)
        h, m = divmod(m, 60)
        print("Running Time", "%d:%02d:%02d" % (h, m, s))
        print("")
    
print('최적화 완료!')

print("")
print("") 

prediction = model 
target = Y

print('예측값 : ', sess.run(prediction, feed_dict = {X : x_train_1, Y: y_train_1, keep_prob : 0.8}))  # 데이터가 안들어 감
print('실제값 : ', sess.run(target, feed_dict = {X : x_train_1, Y: y_train_1, keep_prob : 0.8}))
print("")
print("완료")
print("")
# print('Running Time : %.02f초' % (time.time() - start))
seconds = time.time() - start
m, s = divmod(seconds, 60)
h, m = divmod(m, 60)
print("Running Time", "%d:%02d:%02d" % (h, m, s))

10.0 만 번 :  0.40656245
Running Time 1:24:14

20.0 만 번 :  0.39968032
Running Time 2:48:26

30.0 만 번 :  0.40025857
Running Time 4:10:57

40.0 만 번 :  0.40428078
Running Time 5:32:06

50.0 만 번 :  0.3984198
Running Time 6:53:15

60.0 만 번 :  0.40055117
Running Time 8:15:35

70.0 만 번 :  0.41592264
Running Time 9:39:15

80.0 만 번 :  0.4185595
Running Time 11:02:48

90.0 만 번 :  0.4016472
Running Time 12:25:59

100.0 만 번 :  0.40232623
Running Time 13:47:08

최적화 완료!


예측값 :  [[3.099729 ]
 [2.8356795]
 [1.7965319]
 ...
 [5.0537486]
 [4.901182 ]
 [4.4959455]]
실제값 :  [[2.8332133]
 [3.713572 ]
 [3.4965076]
 ...
 [5.1298985]
 [4.8675346]
 [4.4886365]]

완료

Running Time 13:47:08


####   
####   

## 3. 예측 및 결과 확인

### 1) 예측

In [12]:
predictions = sess.run(model, feed_dict = {X : x_test_1, keep_prob : 0.8})
predictions = np.expm1(predictions) ## == np.exp(predictions) - 1 
predictions

array([[ 14.471618 ],
       [  7.0838985],
       [  3.5641565],
       ...,
       [150.83229  ],
       [119.015915 ],
       [ 80.44408  ]], dtype=float32)

### 2) 결과 제출

In [13]:
submission = pd.read_csv("./bike/sampleSubmission.csv")
submission["count"] = predictions
submission.to_csv("tf226.csv", index=False)

### 3) 결과 보고서 

#### (1) 은닉층 5개 구성 - reduce_mean(tf.square(model - Y)) - train.AdamOptimizer(0.001) - 10 만 번 

In [ ]:
[None, 51] / [51, 256] / [256, 512] / [512, 512] / [512, 256] / [256, 1]  keep_prob : 0.8

#### 결과 : 0.74606로 오히려 은닉층 3개 - 100만 번(0.71249) 보다 결과가 좋지 않음
#### 오차는 더 적어졌으니 실제 test데이터를 돌린 결과 값은 좋지 않음 
#### 과적합이 의심되어 keep_prob : 0.8 설정을 바꾸는 것도 고려해 볼 듯 함.

In [ ]:
9999.0001 만 번 :  0.21423979
19999.0001 만 번 :  0.15759571
29999.0001 만 번 :  0.15061304
39999.0001 만 번 :  0.1446756
49999.0001 만 번 :  0.16366455
59999.0001 만 번 :  0.14469919
69999.0001 만 번 :  0.14174609
79999.0001 만 번 :  0.14150843
89999.0001 만 번 :  0.13308324
99999.0001 만 번 :  0.1427689
Intermediate check : 5326.60초
# 약 1:28:46 분 소요(GPU 0.7 - GTX 1050)

#### (2) 은닉층 5개 구성 - reduce_mean(tf.square(model - Y)) - train.AdamOptimizer(0.001)  -  10 만 번

In [ ]:
[None, 51] / [51, 256] / [256, 512] / [512, 512] / [512, 256] / [256, 1]  :  keep_prob : 0.7  

In [ ]:
1.0 만 번 :  0.22453474
2.0 만 번 :  0.1588468
3.0 만 번 :  0.15966536
4.0 만 번 :  0.18217772
5.0 만 번 :  0.16963457
6.0 만 번 :  0.17445254
7.0 만 번 :  0.19498448
8.0 만 번 :  0.17846932
9.0 만 번 :  0.1943796
10.0 만 번 :  0.17617092
    
Running Time 1:30:41
#약 1시간 30분 소요 (GPU 0.7 - GTX 1050)

#### 결과 : 0.78653. 더 결과값이 안 좋아 졌다.  오차도 크게 낮아지지 않음
#### keep_prob 은 그냥 계속 0.8로 돌리는 것이 나을 듯 함

#### (3) 은닉층 5개 구성 - reduce_mean(tf.square(model - Y)) - train.AdamOptimizer(0.001) - 100 만 번

In [ ]:
[None, 51] / [51, 256] / [256, 512] / [512, 512] / [512, 256] / [256, 1]   keep_prob : 0.8

In [ ]:
10.0 만 번 :  0.13743559
Running Time 2:14:58
20.0 만 번 :  0.12921745
Running Time 3:43:09
30.0 만 번 :  0.13481717
Running Time 5:52:27
40.0 만 번 :  0.15306222
Running Time 7:28:00
50.0 만 번 :  0.16440432
Running Time 9:03:18
60.0 만 번 :  0.17642197
Running Time 10:38:37
70.0 만 번 :  0.17793201
Running Time 12:14:22
        
# 가끔씩 그래픽 사용량이 치솟으면서 컴퓨터가 느려지기도 함. -> 중간에 뻗어버림. 그래픽 사용량 재고
# 10만 번에 대략 2시간 소요 

In [ ]:
10.0 만 번 :  0.12965134
Running Time 1:27:48

20.0 만 번 :  0.13325764
Running Time 2:55:43

30.0 만 번 :  0.14344531
Running Time 4:23:48

40.0 만 번 :  0.14807038
Running Time 5:51:58

# 컴퓨터가 뻗어버림

In [ ]:
10.0 만 번 :  0.16254972
Running Time 1:36:05

20.0 만 번 :  0.15713923
Running Time 3:11:16

30.0 만 번 :  0.1676266
Running Time 4:47:16

40.0 만 번 :  0.17315498
Running Time 6:23:17

50.0 만 번 :  0.17936938
Running Time 7:58:21

60.0 만 번 :  0.17763194
Running Time 9:34:19

70.0 만 번 :  0.17376868
Running Time 11:10:14

80.0 만 번 :  0.17181829
Running Time 12:46:17

90.0 만 번 :  0.2014756
Running Time 14:20:18

100.0 만 번 :  0.18826932
Running Time 15:54:00

-값 나와서 실패  // 0.74644 한 번 더 예측하니 // 0.74931  세번째 예측 : 예측할 때마다 예측 값이 달라짐 

#### (4) 은닉층 7개 구성 - reduce_mean(tf.square(model - Y)) - train.AdamOptimizer(0.001) - 10 만 번

In [ ]:
[None, 51] / [51, 256] / [256, 512] / [512, 512] / [512, 512]  / [512, 512] / [512, 256]   / [256, 1]  :  keep_prob : 0.8  

In [ ]:
1.0 만 번 :  0.18550088
Running Time 0:15:18
2.0 만 번 :  0.15895063
3.0 만 번 :  0.14651746
4.0 만 번 :  0.14147937
5.0 만 번 :  0.13169119
6.0 만 번 :  0.13005805
7.0 만 번 :  0.13422592
8.0 만 번 :  0.13052242
9.0 만 번 :  0.14258853
10.0 만 번 :  0.13765427
Running Time 2:56:02
    
# 약 3시간 소요. (GPU 0.5 - GTX 1050)

#### 결과 : 0.78476으로 더 좋지 않음. 오차율은 떨어졌지만 0.13 아래로 내려가지를 않음.
#### 학습이 오래될 수록 오히려 오차율이 더 안 좋아지는 모습이 보임
#### 학습횟수를 더 늘려보고 마지막으로 은닉층을 더 늘려볼 필요가 있다고 판단됨.

#### (5) 은닉층 5개 구성 - cost = tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))) - train.AdamOptimizer(0.001)

In [ ]:
1000번 : 0.81250 / 
10만번 : 0.86223 // 0.86155  

#### 오래 학습할 수록 실제값이 좋지 않음

#### (6)  은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1))))) - - train.AdamOptimizer(0.001)


In [ ]:
1000번 : 0.78235 /   
10만 번 : 0.81070 /

#### 오차율은 많이 떨어졌으나 결과값은 좋지 않음

#### (7) 은닉층 5개 구성 cost = tf.reduce_mean((tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1))))) 
#### Fail : - 값이나옴

#### (8) 은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(model - Y)) 

In [ ]:
1000번 : 0.76836   가장 최종 결과값이 좋음
5만 번 : -값 나와서 실패(3번 예측했으나 모두)

#### 1000번 학습시 결과값이 매우 좋게 나옴.
#### 다만 5만 번으로 학습을 늘릴 시 -값이 나와서 Fail(3번 예측했으나 모두 -값)

#### ※ 많이 돌릴 수록 오차율이 낮아지다 다시 높아지는 경우가 생김 

#### (8) 은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(model - Y))  - Relu 함수(활성화 함수) 적용 X 

In [ ]:
10만 번 : 0.69327  

In [ ]:
1.0 만 번 :  0.43547434
Running Time 0:08:30

2.0 만 번 :  0.8534407
Running Time 0:17:02

3.0 만 번 :  0.40879902
Running Time 0:25:30

4.0 만 번 :  0.51797825
Running Time 0:33:56

5.0 만 번 :  1.3441521
Running Time 0:42:20

6.0 만 번 :  0.41860524
Running Time 0:50:49

7.0 만 번 :  0.6740765
Running Time 0:59:34

8.0 만 번 :  0.40166765
Running Time 1:08:11

9.0 만 번 :  0.89466834
Running Time 1:16:44

10.0 만 번 :  0.40309197
Running Time 1:25:14

최적화 완료!


예측값 :  [[3.2862644]
 [2.5077457]
 [1.8635235]
 ...
 [5.0984197]
 [4.9109764]
 [4.5148735]]
실제값 :  [[2.8332133]
 [3.713572 ]
 [3.4965076]
 ...
 [5.1298985]
 [4.8675346]
 [4.4886365]]

완료

Running Time 1:25:14

#### (9) 은닉층 5개 구성 - cost = tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1)))) -  Relu함수(활성화함수)적용X
 

In [ ]:
10만 번 : nan / 값이 없음

In [ ]:
예측값 : 
array([[inf],
       [inf],
       [inf],
       ...,
       [inf],
       [inf],
       [inf]], dtype=float32)

#### (10) 은닉층 5개 구성 - cost = tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1))))  - Relu함수적용X

In [ ]:
10만 번 : [inf] / 값이 없음

In [ ]:
array([[inf],
       [inf],
       [inf],
       ...,
       [inf],
       [inf],
       [inf]], dtype=float32)

#### (11) 은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(tf.subtract(tf.log(tf.add(model, 1)), tf.log(tf.add(Y, 1))))) - Relu함수(활성화함수)적용X
 

In [ ]:
10만번 : nan / 값이 없음

#### (12) 은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(model - Y)) - Relu 함수 X - keep_prob : 0.7

In [ ]:
10 만번 : 0.69220  # 현재 가장 좋음

In [ ]:
1.0 만 번 :  0.4931236
Running Time 0:08:37

2.0 만 번 :  0.40457478
Running Time 0:17:15

3.0 만 번 :  0.45067224
Running Time 0:25:55

4.0 만 번 :  0.39891195
Running Time 0:34:37

5.0 만 번 :  0.40459076
Running Time 0:43:38

6.0 만 번 :  0.40142223
Running Time 0:52:38

7.0 만 번 :  0.39655456
Running Time 1:01:38

8.0 만 번 :  0.4069065
Running Time 1:10:41

9.0 만 번 :  0.3995612
Running Time 1:19:43

10.0 만 번 :  0.4067454
Running Time 1:28:40

최적화 완료!


예측값 :  [[3.2910514]
 [2.7232914]
 [2.5623527]
 ...
 [5.041258 ]
 [4.8719373]
 [4.4741297]]
실제값 :  [[2.8332133]
 [3.713572 ]
 [3.4965076]
 ...
 [5.1298985]
 [4.8675346]
 [4.4886365]]

완료

Running Time 1:28:40

#### (13) 은닉층 5개 -  cost = tf.reduce_mean(tf.square(model - Y)) - Relu 함수 X - keep_prob : 0.7 -  random_uniform

In [ ]:
10만 번 : nan / 값이 없음

#### (14) 은닉층 5개 구성 - cost = tf.reduce_mean(tf.square(model - Y))  - Relu 함수(활성화 함수) 적용 X 

100만 번 : 0.81796 // 0.81641 (예측만 두 번)

In [ ]:
10.0 만 번 :  0.40656245
Running Time 1:24:14

20.0 만 번 :  0.39968032
Running Time 2:48:26

30.0 만 번 :  0.40025857
Running Time 4:10:57

40.0 만 번 :  0.40428078
Running Time 5:32:06

50.0 만 번 :  0.3984198
Running Time 6:53:15

60.0 만 번 :  0.40055117
Running Time 8:15:35

70.0 만 번 :  0.41592264
Running Time 9:39:15

80.0 만 번 :  0.4185595
Running Time 11:02:48

90.0 만 번 :  0.4016472
Running Time 12:25:59

100.0 만 번 :  0.40232623
Running Time 13:47:08

최적화 완료!


예측값 :  [[3.099729 ]
 [2.8356795]
 [1.7965319]
 ...
 [5.0537486]
 [4.901182 ]
 [4.4959455]]
실제값 :  [[2.8332133]
 [3.713572 ]
 [3.4965076]
 ...
 [5.1298985]
 [4.8675346]
 [4.4886365]]

완료

Running Time 13:47:08

####   
####  

 ## 4. Bike Sharing Demand TensorFlow modeling 결론

#### ◎ 은닉층은 5개, 손실함수는 cost = tf.reduce_mean(tf.square(model - Y))가 가장 좋은 결과를 보인다.
#### ◎ 활성화 함수인 ReLu함수를 적용하지 않은 것이 더 낫다.
#### ◎ 학습 횟수는 10만 번에서 준수하다. 100만 번으로 많이 했는데 실제 결과값은 더 안 좋아졌다.



####    
####    
####   